Funny python functions in other files don't get refreshed unless restart kernel, below magic is witchcraft that kinda fixes problem.

In [4]:
%load_ext autoreload
%autoreload 2

# Custom script imports for compartmentalization 

In [ ]:
import visualization, preprocess, metrics, naive_autoencoder, analyze_compression

2025-04-15 06:10:14.655498: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 06:10:14.689699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-15 06:10:15.307107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Check if Tensorflow sees your GPU and has CUDA
### Unnecessary unless you want to run large models on your GPU

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Built with GPU support:", tf.test.is_built_with_gpu_support())
print("GPU available:", tf.config.list_physical_devices('GPU'))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4052618660117580701
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14211350528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11080056222649611341
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]
Built with CUDA: True
Built with GPU support: True
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-04-15 05:40:21.665640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 05:40:21.781556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 05:40:21.781591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 05:40:22.032562: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-15 05:40:22.032609: I tensorflow/compile

# Autoencoder Model
### Ensure when creating new model that you return the autoencoder itself as well as latent space encoder.

In [10]:
sequence_length = 3
height = 64
width = 64
channels = 3

autoencoder, encoder = naive_autoencoder.build_conv_lstm_autoencoder(
    sequence_length=sequence_length,
    height=height,
    width=width,
    channels=channels
)

autoencoder.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 64, 64, 3)]    0         
                                                                 
 time_distributed (TimeDistr  (None, 3, 64, 64, 32)    896       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 3, 64, 64, 64)    18496     
 tributed)                                                       
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 3, 64, 64, 64)     295168    
                                                                 
 time_distributed_2 (TimeDis  (None, 3, 64, 64, 64)    36928     
 tributed)                                                       
                                                             

# Preprocess
### Resizes, normalizes, and formats data-splitting into training and validation sets

In [ ]:
video_folder = "extracted_videos"

# Preprocess the videos
video_data, video_filenames = preprocess_videos_with_mapping(
    video_folder=video_folder,
    sequence_length=sequence_length,
    height=height,
    width=width,
    channels=channels
)

# Split the data into training and validation sets
split_index = int(0.8 * len(video_data))
train_data = video_data[:split_index]
val_data = video_data[split_index:]

# Reshape the training and validation data to match the input shape of the autoencoder
train_data = train_data.reshape((-1, sequence_length, height, width, channels))
val_data = val_data.reshape((-1, sequence_length, height, width, channels))

print("Train data shape:", train_data.shape)  # Expected: (num_samples, sequence_length, height, width, channels)
print("Validation data shape:", val_data.shape)


Train data shape: (37254, 3, 64, 64, 3)
Validation data shape: (9314, 3, 64, 64, 3)


# ONLY RUN IF YOU'RE NATHAN OR WANT TO WASTE TIME

Or I suppose if you make a small baby model

In [ ]:
# Train the autoencoder
autoencoder.fit(
    train_data, train_data,
    validation_data=(val_data, val_data),
    epochs=10,
    batch_size=4
)

Train data shape: (37146, 3, 64, 64, 3)
Validation data shape: (9287, 3, 64, 64, 3)
Epoch 1/10


2025-04-03 13:52:01.078876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-04-03 13:52:01.282855: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2025-04-03 13:52:01.282889: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2025-04-03 13:52:01.283137: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:317] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2025-04-03 13:52:01.815474: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fd9ee002930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-03 13:52:01.815508: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecuto

9287/9287 [==============================] - 382s 41ms/step - loss: 7.7424e-04 - val_loss: 3.6359e-04
Epoch 2/10
9287/9287 [==============================] - 361s 39ms/step - loss: 1.4463e-04 - val_loss: 5.4172e-05
Epoch 3/10
9287/9287 [==============================] - 360s 39ms/step - loss: 9.0927e-05 - val_loss: 3.7964e-05
Epoch 4/10
9287/9287 [==============================] - 363s 39ms/step - loss: 7.0415e-05 - val_loss: 2.9867e-04
Epoch 5/10
9287/9287 [==============================] - 362s 39ms/step - loss: 6.1055e-05 - val_loss: 4.0588e-05
Epoch 6/10
9287/9287 [==============================] - 360s 39ms/step - loss: 5.1567e-05 - val_loss: 1.7583e-05
Epoch 7/10
9287/9287 [==============================] - 361s 39ms/step - loss: 4.6759e-05 - val_loss: 1.6843e-05
Epoch 8/10
9287/9287 [==============================] - 361s 39ms/step - loss: 4.2643e-05 - val_loss: 3.3355e-05
Epoch 9/10
9287/9287 [==============================] - 360s 39ms/step - loss: 3.9196e-05 - val_loss: 5.795

In [ ]:
# If have multiple autoencoders trained, can change name to load different one
encoder_file = "video_autoencoder.keras"

## SAVES ENCODER. ONLY USE IF YOU'VE JUST TRAINED A MODEL YOU WANT TO KEEP

In [ ]:
from tensorflow.keras.models import save_model

autoencoder.save(encoder_file)

## LOADS MODEL, PROBABLY ALWAYS DO THIS

In [ ]:
from tensorflow.keras.models import load_model

autoencoder = load_model(encoder_file)

# It's the metrics, wtf else is there to say

In [ ]:
metrics.compute_video_metrics(video_path="comparison_celebv_-_Bdf9C0SAU_0.avi", frame_width=64)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nchanak/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


{'psnr': 39.454144, 'ssim': 0.9577416, 'lpips': 0.011201685887672716}

# Compression
### Yes very bad compression rn cuz latent space huge

In [26]:
import analyze_compression

# Assume you've already run
# autoencoder, encoder = build_conv_lstm_autoencoder(...)
# autoencoder.load_weights(...)
# video_data, video_filenames = preprocess_videos_with_mapping(...)

result = analyze_compression.analyze_compression_from_video_file(
    video_name="celebv_-3KCgSpt3hU_5.mp4",
    autoencoder=autoencoder,
    encoder=encoder,
    sequence_length=3,
    height=64,
    width=64
)

[▶] Processing 82 sequences from: celebv_-3KCgSpt3hU_5.mp4
82/82 [==============================] - 0s 5ms/step

📦 Compression Results for 'celebv_-3KCgSpt3hU_5.mp4':
Raw 64x64 input size:    558.52 KB
Latent compressed size:  148215.69 KB
Estimated compression:   0.00x


requires: conda install -c conda-forge ffmpeg

alternatively: sudo apt install ffmpeg


In [25]:
play_video = "celebv_-3KCgSpt3hU_5.mp4"

comparison_video = visualization.generate_video_comparison_by_name(
    video_name=play_video,
    autoencoder=autoencoder,
    sequence_length=sequence_length,
    height=height,
    width=width
)
mp4_file = visualization.convert_avi_to_mp4(comparison_video, scale=4)
visualization.display_video_inline(mp4_file, width=512)


3/3 [==============================] - 0s 170ms/step
[✓] Comparison saved: comparison_celebv_-3KCgSpt3hU_5.avi
[▶] Converting comparison_celebv_-3KCgSpt3hU_5.avi → comparison_celebv_-3KCgSpt3hU_5.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab